In [1]:

'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import RMSprop


from keras import backend as K
from keras import layers
import numpy as np
import tensorflow as tf


class PlasticDense(layers.Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        #self.batch_size = batch_size
        super(PlasticDense, self).__init__(**kwargs)
        
    def build(self, input_shape):
        
        #Define set of traditional weights
        self.w = self.add_weight(name='w', 
                                 shape=(input_shape[1], self.output_dim),
                                 initializer='uniform',
                                 trainable=True)
        
        #Define our plasticity coefficient
        self.alpha = self.add_weight(name='alpha', 
                              shape=(1, 1),
                              initializer='uniform',
                              trainable=True)
        
        
        #The Hebbian trace
        self.hebb = self.add_weight(name='hebb', 
                              shape=(input_shape[1], self.output_dim),
                              initializer='zeros',
                              trainable=False)
        
        #Step size will be optimized
        self.eta = self.add_weight(name='eta', 
                                      shape=(1, 1),
                                      initializer='uniform',
                                      trainable=False)
        super(PlasticDense, self).build(input_shape)

        
        #yout = F.tanh( yin.mm(self.w + torch.mul(self.alpha, hebb)) + input )
        #hebb = (1 - self.eta) * hebb + self.eta * torch.bmm(yin.unsqueeze(2), yout.unsqueeze(1))[0] # bmm here is used to implement an outer product between yin and yout, with the help of unsqueeze (i.e. added empty dimensions)
        #return yout, hebb

    def call(self, x):
        
        #X (layer input)     : shape(?, INPUT_DIM)
        #W                   : shape(INPUT_DIM, OUTPUT_DIM)
        #hebb                : shape(INPUT_DIM, OUTPUT_DIM)
        #Y (layer output)    : shape(?, OUTPUT_DIM)
        #ETA                 : scalar (one per layer)
        
        #yout = K.maximum(0.0, np.add((K.dot(self.y, np.add(K.dot(self.alpha, K.transpose(self.hebb)), self.w))), x))
        #hebb = (1 - 0.01) * self.hebb + 0.01 * K.dot(self.y, yout)
        #yout = K.maximum(0.0, np.add(self.y * np.add(self.alpha * self.hebb, self.w), x))
        
        #y = K.dot(x, self.w)
        #y = F.tanh( yin.mm(self.w + torch.mul(self.alpha, hebb)) + yin )
        #plastic_y = self.alpha * (K.dot(x, self.hebb)) 
        #self.hebb = (1 - self.eta) * self.hebb + self.eta * torch.bmm(yin.unsqueeze(2), yout.unsqueeze(1))[0]
        #model_out = K.maximum(0.0, y + plastic_y)
        
        #Hebbian update - option 1
        #self.hebb = self.eta * K.dot(x, model_out) + (1 - self.eta) * self.hebb
        #print(self.hebb)
        
        #Hebbian update - option 2
        #self.hebb +=self.eta * K.dot(model_out, (x - (K.dot(model_out, self.hebb))))
        
        y = K.dot(x, self.w)
        plastic_y = self.alpha * (K.dot(x, self.hebb))   
        model_out = K.maximum(0.0, y + plastic_y)
        
        #Hebbian update - option 1
        self.hebb = self.eta * K.dot(x, model_out) + (1 - self.eta) * self.hebb
        print(self.hebb)

        return model_out
        #return K.maximum(0.0, y)
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

batch_size = 128
num_classes = 10
epochs = 16

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

def testClassic(num_neurons, num_layers, epochs):
    print("Classic: n: {} l: {} e: {}".format(num_neurons, num_layers, epochs))
    classifier = Sequential()
    classifier.add(Dense(output_dim = num_neurons, init = 'uniform', activation = 'relu', input_dim = 11))
    for l in range(num_layers - 1):
        classifier.add(Dense(output_dim = num_neurons, init = 'uniform', activation = 'relu'))
    classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    train_history = classifier.fit(x_train, y_train, batch_size = 128, nb_epoch = epochs)
    
    loss_history = train_history.history["loss"]
    np.savetxt("results/n_{}_l_{}_e_{}_loss_classic.txt".format(num_neurons, num_layers, epochs), 
                  np.array(loss_history), delimiter=",")
    
    acc_history = train_history.history["acc"]
    np.savetxt("results/n_{}_l_{}_e_{}_acc_classic.txt".format(num_neurons, num_layers, epochs), 
                  np.array(acc_history), delimiter=",")
    # TODO Append test prediction results to file or to its own file
    #y_pred = classifier.predict(X_test)
    #return (y_pred > 0.5)


def testPlastic(num_neurons, num_layers, epochs):
    print("Plastic: n: {} l: {} e: {}".format(num_neurons, num_layers, epochs))
    classifier = Sequential()
    classifier.add(PlasticDense(num_neurons, input_shape=(11,)))
    for l in range(num_layers - 1):
        classifier.add(PlasticDense(num_neurons))
    classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    train_history = classifier.fit(x_train, y_train, batch_size = 128, nb_epoch = epochs)
        
    loss_history = train_history.history["loss"]
    np.savetxt("results/n_{}_l_{}_e_{}_loss_plastic.txt".format(num_neurons, num_layers, epochs), 
                  np.array(loss_history), delimiter=",")
    
    acc_history = train_history.history["acc"]
    np.savetxt("results/n_{}_l_{}_e_{}_acc_plastic.txt".format(num_neurons, num_layers, epochs), 
                  np.array(acc_history), delimiter=",")
    # TODO Append test prediction results to file or to its own file
   # y_pred = classifier.predict(X_test)
   # return (y_pred > 0.5)
    
    
def extensive_test():
    neuron_combos = [4, 6, 8, 12, 16]
    layer_combos = [2, 4, 8, 16]
    epochs = 200
    
    i = 0
    for n in neuron_combos:
        for l in layer_combos:
            testClassic(n, l, epochs)
            testPlastic(n, l, epochs)
            i += 1
            print("Try: {}/{}".format(i, len(neuron_combos)*len(layer_combos)))

#extensive_test()
y_pred = testClassic(4, 4, 2)
y_pred = testPlastic(4, 4, 20)

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
#score = model.evaluate(x_test, y_test, verbose=0)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


60000 train samples
10000 test samples
Classic: n: 4 l: 4 e: 2
Instructions for updating:
keep_dims is deprecated, use keepdims instead


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:125: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="relu", input_dim=11, units=4)`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:127: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=4, activation="relu", kernel_initializer="uniform")`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:128: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, activation="sigmoid", kernel_initializer="uniform")`
/home/mun/.local/lib/python2.7/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


ValueError: Error when checking input: expected dense_1_input to have shape (None, 11) but got array with shape (60000, 784)